In [55]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# modulo para cargar archivos en formato MATLAB
# from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

import pandas as pd

In [56]:
# Leemos el dataset
data = pd.read_csv('star_classification.csv')
# Mostramos el dataset
display(data)

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.237679e+18,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,301,2,581,1.055431e+19,GALAXY,0.000000,9374,57749,438
99996,1.237679e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,301,1,289,8.586351e+18,GALAXY,0.404895,7626,56934,866
99997,1.237668e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,301,4,308,3.112008e+18,GALAXY,0.143366,2764,54535,74
99998,1.237661e+18,212.268621,46.660365,25.35039,21.63757,19.91386,19.07254,18.62482,3650,301,4,131,7.601080e+18,GALAXY,0.455040,6751,56368,470


In [57]:
# Eliminamos las columnas que no nos serviran y llevamos la columna 'class' al final del dataset
data = data.drop(['obj_ID', 'run_ID', 'rerun_ID', 'field_ID', 'spec_obj_ID', 'fiber_ID'], axis=1)
columnas = data.columns.tolist()
columnas.remove("class")
columnas.append("class")
data = data[columnas]
display(data)

,alpha,delta,u,g,r,i,z,cam_col,redshift,plate,MJD,class
0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,2,0.634794,5812,56354,GALAXY
1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,5,0.779136,10445,58158,GALAXY
2,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,2,0.644195,4576,55592,GALAXY
3,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,3,0.932346,9149,58039,GALAXY
4,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,3,0.116123,6121,56187,GALAXY
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,2,0.000000,9374,57749,GALAXY
99996,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,1,0.404895,7626,56934,GALAXY
99997,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,4,0.143366,2764,54535,GALAXY
99998,212.268621,46.660365,25.35039,21.63757,19.91386,19.07254,18.62482,4,0.455040,6751,56368,GALAXY


In [58]:
from sklearn.preprocessing import LabelEncoder
#Transformacion de columnas categoricas a numericas
#selecciona las columnas del dataset de tipo object
columnas_categoricas = data.select_dtypes(include=['object']).columns
#El for recorrera cada columna categorica
for columna in columnas_categoricas:
  #El LabelEncoder se utiliza para codificar las etiquetas categóricas como valores numéricos
  le = LabelEncoder()
  #aplica la transformación a la columna actual
  #El método fit_transform del LabelEncoder ajusta el codificador a los datos de la columna
  # y transforma las etiquetas categóricas en valores numéricos
  data[columna] = le.fit_transform(data[columna])

In [59]:
display(data)

,alpha,delta,u,g,r,i,z,cam_col,redshift,plate,MJD,class
0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,2,0.634794,5812,56354,0
1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,5,0.779136,10445,58158,0
2,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,2,0.644195,4576,55592,0
3,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,3,0.932346,9149,58039,0
4,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,3,0.116123,6121,56187,0
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,2,0.000000,9374,57749,0
99996,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,1,0.404895,7626,56934,0
99997,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,4,0.143366,2764,54535,0
99998,212.268621,46.660365,25.35039,21.63757,19.91386,19.07254,18.62482,4,0.455040,6751,56368,0


In [60]:
# Inicializamos el valor de num_labels en 3 porque y tiene 3 clases
num_labels = 3
# Asignamos las columnas a X y y
X = data.iloc[:, :11]
y = data.iloc[:, 11]
# Aisgnamos el valor de m con la cantidad de filas que hay en y
m = y.size

In [61]:
# Imprimimos X
print(X)

            alpha      delta         u         g         r         i  \
0      135.689107  32.494632  23.87882  22.27530  20.39501  19.16573   
1      144.826101  31.274185  24.77759  22.83188  22.58444  21.16812   
2      142.188790  35.582444  25.26307  22.66389  20.60976  19.34857   
3      338.741038  -0.402828  22.13682  23.77656  21.61162  20.50454   
4      345.282593  21.183866  19.43718  17.58028  16.49747  15.97711   
...           ...        ...       ...       ...       ...       ...   
99995   39.620709  -2.594074  22.16759  22.97586  21.90404  21.30548   
99996   29.493819  19.798874  22.69118  22.38628  20.45003  19.75759   
99997  224.587407  15.700707  21.16916  19.26997  18.20428  17.69034   
99998  212.268621  46.660365  25.35039  21.63757  19.91386  19.07254   
99999  196.896053  49.464643  22.62171  21.79745  20.60115  20.00959   

              z  cam_col  redshift  plate    MJD  
0      18.79371        2  0.634794   5812  56354  
1      21.61427        5  0.77913

In [62]:
# Imprimimos y
print(y)

0        0
1        0
2        0
3        0
4        0
        ..
99995    0
99996    0
99997    0
99998    0
99999    0
Name: class, Length: 100000, dtype: int32


In [63]:
def  featureNormalize(X): # define una función llamada featureNormalize que toma una matriz X como argumento de entrada
    X_norm = X.copy() # se crea una copia de la matriz de entrada X llamada X_norm
    # Se crean dos matrices mu y sigma, cada una con un tamaño igual al número de columnas de X
    mu = np.zeros(X.shape[1]) 
    sigma = np.zeros(X.shape[1])

    # Aquí se calculan las medias y desviaciones estándar de cada columna de X utilizando las funciones np.mean y np.std de NumPy
    # El argumento axis=0 indica que los cálculos deben realizarse a lo largo de las columnas
    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma # se realiza la normalización de características

    return X_norm, mu, sigma # la funcion devuelve tres valores: X_norm, mu, sigma

In [64]:
# llamamos a la funcion featureNormalize, enviando como parametro X
# que nos devuelve los valores de X_norm, mu, sigma
X_norm, mu, sigma = featureNormalize(X)

In [65]:
# se asignan los valores del número de filas (m) y el número de columnas (n)
# de la matriz X a las variables m y n, respectivamente
m, n = X.shape
X = X_norm # la variable X se actualiza y se le asigna la matriz X_norm

In [66]:
# se calcula la función sigmoide de z
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

In [67]:
def lrCostFunction(theta, X, y, lambda_): #define una función llamada lrCostFunction que toma cuatro argumentos

    m = y.size # asignamos a m la cantidad de filas que tiene y

    # verifica si el tipo de datos de y es booleano. Si es así, se convierte
    # a tipo entero mediante el método astype(int) de NumPy
    if y.dtype == bool:
        y = y.astype(int)

    # Se inicializan las variables J y grad con valores iniciales de cero
    J = 0
    grad = np.zeros(theta.shape)

    # Aquí se calcula la hipótesis del modelo, h, aplicando la función sigmoide
    h = sigmoid(X.dot(theta.T))

    # Se crea una copia de los parámetros theta llamada temp. Luego, se establece el primer elemento de temp en cero
    temp = theta
    temp[0] = 0

    # Esta línea calcula el costo J
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    # El primer término calcula el gradiente sin regularización, mientras que el segundo término aplica la regularización
    # grad = (1 / m) * (h - y).dot(X)
    # grad = grad + (lambda_ / m) * temp

    grad = (1 / m) * (h - y).dot(X) + (lambda_ / m) * temp

    return J, grad

In [68]:
def oneVsAll(X, y, num_labels, lambda_):
    # m igual a filas de X, n igual a las columnas de X
    m, n = X.shape
    
    # Se crea una matriz all_theta de tamaño (num_labels, n + 1) con valores iniciales de cero
    all_theta = np.zeros((num_labels, n + 1))

    # Aquí se agrega una columna de unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    # Este bucle itera sobre las etiquetas de clase
    for c in np.arange(num_labels):
        # Dentro del bucle, se inicializan los parámetros initial_theta con ceros
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        # se utiliza la función optimize.minimize de SciPy para minimizar la función de costo lrCostFunction
        # Se proporcionan los argumentos X, (y == c) (que crea un vector de booleanos que indica si
        # cada muestra pertenece a la clase c o no), lambda_, jac=True para indicar que se proporcionará
        # el gradiente en lrCostFunction, method='CG' para utilizar el método de optimización de gradiente
        # conjugado, y options para configurar el número máximo de iteraciones en 50.
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)
        # Se asigna el vector de parámetros óptimos res.x a la fila c de la matriz all_theta.
        all_theta[c] = res.x

    return all_theta

In [69]:
# asignamos a lambda_ el valor de 0.1
lambda_ = 0
# llamamos a la funcion oneVsAll para obtener all_theta
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta.shape) # (3, 12)

C:\Users\hlope\AppData\Local\Temp\ipykernel_18964\1691343629.py:3: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))
C:\Users\hlope\AppData\Local\Temp\ipykernel_18964\2736554574.py:22: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
C:\Users\hlope\AppData\Local\Temp\ipykernel_18964\1691343629.py:3: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))
C:\Users\hlope\AppData\Local\Temp\ipykernel_18964\2736554574.py:22: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
C:\Users\hlope\AppData\Local\Temp\ipykernel_18964\1691343629.py:3: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))
C:\Users\hlope\AppData\Local\Temp\ipykernel_18964\2736554574.py:22: RuntimeWarning: divide 

(3, 12)


C:\Users\hlope\AppData\Local\Temp\ipykernel_18964\2736554574.py:22: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))


In [70]:
# imprimimos los valores de all_theta
print(all_theta)

[[ 4.43494960e-01  2.25867204e-02 -8.05410058e-02  9.40575198e+00
   8.59528250e+00  2.54737467e+00 -3.41177753e+00 -2.15805250e+00
  -6.74777205e-02 -2.88976727e-01  1.02227334e+00 -1.09265155e+00]
 [-2.27452228e+00  4.84043207e-02  3.17589895e-01 -7.04569773e+00
  -9.92745004e-01 -3.65365442e+00  2.96439217e+00  7.99413349e+00
   1.93840274e-02  4.73572487e+00 -3.00744669e-01  9.88318977e-02]
 [-2.74865844e+01 -1.16818965e-01 -9.52959362e-02  2.63896753e-01
  -7.73615891e-01  4.17863178e-01  8.29874284e-02  5.15024641e-01
   8.87883576e-02 -3.81281815e+01 -2.36738670e+00  2.53544096e+00]]


In [71]:
def predictOneVsAll(all_theta, X):
    # Aquí se obtiene el número de muestras m de X y el número de clases num_labels del clasificador.
    m = X.shape[0];
    num_labels = all_theta.shape[0]
    
    # Se crea un vector p de tamaño m inicializado con ceros
    p = np.zeros(m)
    
    # Aquí se agrega una columna de unos a la matriz X de las muestras
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    # Se calcula el producto de la matriz X y los parámetros all_theta transpuestos,
    # y se aplica la función sigmoide a la salida. Luego, se utiliza la función np.argmax
    # de NumPy para obtener el índice del valor máximo a lo largo del eje axis=1, que corresponde
    # a la clase con la mayor probabilidad para cada muestra. Estos índices se asignan al vector
    # p como las predicciones de clase.
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)
 
    return p

In [72]:
# imprime la forma de la matriz X
print(X.shape)
# Aquí se llama a la función predictOneVsAll para realizar la predicción de clases utilizando
# los parámetros all_theta y la matriz de características X
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
# Aquí se selecciona un subconjunto de las muestras de entrenamiento original X utilizando
# el índice de fila 10 a 149 (excluyendo la fila 150). Los datos se copian en la variable XPrueba.
XPrueba = X.iloc[10:150, :].copy()
# imprime la forma de la matriz XPrueba
print(XPrueba.shape)
# Aquí se agrega una columna de unos a la matriz XPrueba de las muestras de prueba
XPrueba = np.concatenate([np.ones((140, 1)), XPrueba], axis=1)
print(XPrueba.shape)
# Se calcula la predicción de clase para las muestras de prueba en XPrueba. Se realiza
# la combinación lineal de las características y los parámetros utilizando el producto
# punto (dot) y se aplica la función sigmoide a la salida. Luego, se utiliza la función
# np.argmax para obtener el índice del valor máximo a lo largo del eje axis=1
# (es decir, la clase con la mayor probabilidad) para cada muestra de prueba.
# El resultado se asigna a la variable p.
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print(p)
# Aquí se imprime el subconjunto correspondiente de las etiquetas de clase y para las muestras de prueba seleccionadas
# print(y[10:150])

(100000, 11)
Precision del conjuto de entrenamiento: 92.83%
(140, 11)
(140, 12)
[0 0 2 0 0 0 2 1 2 0 0 2 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 2 0 2 0 0 0 0 2 0
 1 1 0 0 1 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 2 0 0 0 0 1 0 2 1
 0 1 0 1 0 2 2 0 0 0 0 0 2 2 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 1 2 0
 1 0 2 0 0 0 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1 0 0 0 0 0 0]


C:\Users\hlope\AppData\Local\Temp\ipykernel_18964\1691343629.py:3: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))
